In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2
from keras import backend as K
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Embedding,SpatialDropout1D,LSTM
from keras.utils import to_categorical
import copy
from keras.callbacks import ModelCheckpoint
import re
from google.colab import drive
from sklearn.preprocessing import LabelBinarizer 
drive.mount("/content/drive")

KeyboardInterrupt: 

In [0]:
#1c(i)
f1=open("/content/drive/My Drive/BertrandRusell.txt", 'rb')
Br_text=(f1.read().decode().replace("\n","").replace("\r","").replace("\t",""))
print("Text from reading the books: ")
print(Br_text[0:1000])

Text from reading the books: 
OUR KNOWLEDGE OF THE  EXTERNAL WORLD  AS A FIELD FOR SCIENTIFIC METHOD  IN PHILOSOPHYBY THE SAME AUTHOR  INTRODUCTION TO MATHEMATICAL PHILOSOPHY  Second Edition.  Demy 8vo, 12s. 6d. net.  THE ANALYSIS OF MIND  Demy 8vo, 16s. net.  PRINCIPLES OF SOCIAL RECONSTRUCTION  Seventh Impression.  Cr. 8vo, 5s. net; Limp, 3s. 6d. net.  ROADS TO FREEDOM: SOCIALISM, ANARCHISM AND SYNDICALISM  Fourth Impression.  Cr. 8vo, 5s. net; Limp, 3s. 6d. net.  THE PRACTICE AND THEORY OF BOLSHEVISM  Second Impression.  Cr. 8vo, 6s. net.  OUR KNOWLEDGE OF  THE EXTERNAL WORLD  AS A FIELD FOR SCIENTIFIC METHOD  IN PHILOSOPHY  BY  BERTRAND RUSSELL, F.R.S  LONDON: GEORGE ALLEN & UNWIN LTD  RUSKIN HOUSE, 40 MUSEUM STREET, W.C. 1  First published in 1914 by  The Open Court Publishing Company  Reissued by George Allen & Unwin Ltd.  1922PREFACEThe following lectures[1] are an attempt to show, by means of examples,the nature, capacity, and limitations of the logical-analytic method inphilos

In [0]:
#1c(ii)
Br_text=Br_text.lower()
Br_text=re.sub('[^a-zA-Z0-9.,_+;\s]+', '', Br_text)
dictionary = dict()
max1=0
min1=1000
for ch in list(set(Br_text)):
    dictionary[ch] = ord(ch)
    if(ord(ch)>max1):
        max1=ord(ch)
    if(ord(ch)<min1):
        min1=ord(ch)

for k,v in dictionary.items():
    dictionary[k]=round((v-min1)/(max1-min1),3)
    
print(dictionary)


{'h': 0.8, ';': 0.3, '4': 0.222, 'n': 0.867, 'l': 0.844, 'o': 0.878, 'x': 0.978, 's': 0.922, 'b': 0.733, '8': 0.267, '9': 0.278, '3': 0.211, 'u': 0.944, '.': 0.156, 'g': 0.789, 'c': 0.744, '2': 0.2, '6': 0.244, 'j': 0.822, ',': 0.133, 'e': 0.767, '+': 0.122, 'f': 0.778, ' ': 0.0, 'r': 0.911, 'd': 0.756, 'v': 0.956, '1': 0.189, '5': 0.233, 'y': 0.989, 'm': 0.856, '_': 0.7, 'p': 0.889, 't': 0.933, 'z': 1.0, 'k': 0.833, 'i': 0.811, '7': 0.256, 'w': 0.967, '0': 0.178, 'q': 0.9, 'a': 0.722}


In [0]:
#1c(iii)
print("Window size : 100")

Window size : 100


In [0]:
#1c(iv)
X_train=[]
y_train=[]
y_train_temp=[]
for i in range(0,len(Br_text)):
    if((i+99)>len(Br_text)-1):
        break
    temp=list(Br_text[i:i+99])
    res=[]
    for k in temp:
        res.append(dictionary[k])
    X_train.append(res) 
    y_train.append(dictionary[Br_text[i+99]])
    y_train_temp.append(ord(Br_text[i+99]))
    i=i+1
print("Input train data fed into the network:")
print(X_train[0:5])
print(" ")
print("Input train labels:")
print(y_train_temp[0:5])


Input train data fed into the network:
[[0.878, 0.944, 0.911, 0.0, 0.833, 0.867, 0.878, 0.967, 0.844, 0.767, 0.756, 0.789, 0.767, 0.0, 0.878, 0.778, 0.0, 0.933, 0.8, 0.767, 0.0, 0.0, 0.767, 0.978, 0.933, 0.767, 0.911, 0.867, 0.722, 0.844, 0.0, 0.967, 0.878, 0.911, 0.844, 0.756, 0.0, 0.0, 0.722, 0.922, 0.0, 0.722, 0.0, 0.778, 0.811, 0.767, 0.844, 0.756, 0.0, 0.778, 0.878, 0.911, 0.0, 0.922, 0.744, 0.811, 0.767, 0.867, 0.933, 0.811, 0.778, 0.811, 0.744, 0.0, 0.856, 0.767, 0.933, 0.8, 0.878, 0.756, 0.0, 0.0, 0.811, 0.867, 0.0, 0.889, 0.8, 0.811, 0.844, 0.878, 0.922, 0.878, 0.889, 0.8, 0.989, 0.733, 0.989, 0.0, 0.933, 0.8, 0.767, 0.0, 0.922, 0.722, 0.856, 0.767, 0.0, 0.722, 0.944], [0.944, 0.911, 0.0, 0.833, 0.867, 0.878, 0.967, 0.844, 0.767, 0.756, 0.789, 0.767, 0.0, 0.878, 0.778, 0.0, 0.933, 0.8, 0.767, 0.0, 0.0, 0.767, 0.978, 0.933, 0.767, 0.911, 0.867, 0.722, 0.844, 0.0, 0.967, 0.878, 0.911, 0.844, 0.756, 0.0, 0.0, 0.722, 0.922, 0.0, 0.722, 0.0, 0.778, 0.811, 0.767, 0.844, 0.756, 0.0, 

In [0]:
#1c(v)
y_train1 = to_categorical(y_train_temp)
label_binarizer = LabelBinarizer()
label_binarizer.fit(y_train_temp)
label_classes=label_binarizer.classes_
y_train1=label_binarizer.transform(y_train_temp)
print("One hot encoded labels: ")
print(y_train1[0:5])

One hot encoded labels: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]]


In [51]:
#1c(vi,vii,viii)
model_lstm = Sequential()
model_lstm.add(LSTM(256,input_shape=( 99, 1)))
model_lstm.add(Dense(42, activation = 'softmax'))
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)
filepath="weights_new.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]
model_lstm.save_weights('/content/drive/My Drive/Colab Notebooks/HW7Weights1/BertrandRusell/'+filepath)
history = model_lstm.fit(
    np.array(X_train).reshape(len(X_train),99,1),
    np.array(y_train1),
    epochs = 30,
    batch_size=2000,
    callbacks=[checkpoint]
)

Epoch 1/30
1739871/1739871 [==============================] - 492s 283us/step - loss: 2.8466 - acc: 0.1846

Epoch 00001: loss improved from inf to 2.84664, saving model to weights_new.best.hdf5
Epoch 2/30
1739871/1739871 [==============================] - 493s 283us/step - loss: 2.7547 - acc: 0.2059

Epoch 00002: loss improved from 2.84664 to 2.75468, saving model to weights_new.best.hdf5
Epoch 3/30
1739871/1739871 [==============================] - 492s 283us/step - loss: 2.6981 - acc: 0.2168

Epoch 00003: loss improved from 2.75468 to 2.69814, saving model to weights_new.best.hdf5
Epoch 4/30
1739871/1739871 [==============================] - 492s 283us/step - loss: 2.6469 - acc: 0.2311

Epoch 00004: loss improved from 2.69814 to 2.64692, saving model to weights_new.best.hdf5
Epoch 5/30
1739871/1739871 [==============================] - 493s 283us/step - loss: 2.6022 - acc: 0.2452

Epoch 00005: loss improved from 2.64692 to 2.60222, saving model to weights_new.best.hdf5
Epoch 6/30
173

In [39]:
print("Cross-entropy loss, or log loss, measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label. So predicting a probability of .012 when the actual observation label is 1 would be bad and result in a high loss value. A perfect model would have a log loss of 0.")

Cross-entropy loss, or log loss, measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label. So predicting a probability of .012 when the actual observation label is 1 would be bad and result in a high loss value. A perfect model would have a log loss of 0.


In [0]:
#1c(ix)
print("Number of epochs: 30")

Number of epochs: 30


In [0]:
#1c(x,xi)
test_data="There are those who take mental phenomena naively, just as they \
would physical phenomena. This school of psychologists tends not to \
emphasize the object."
test_data=test_data.lower()
test_data=re.sub('[^a-zA-Z0-9.,_+;\s]+', '', test_data)
#print(test_data)


model_lstm.load_weights('/content/drive/My Drive/Colab Notebooks/HW7Weights1/BertrandRusell/'+filepath)
model_lstm.compile(
    loss='categorical_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)
final_res=""
for l in range(0,1000):
  X_test=[]
  test_data=test_data[-99:]
  #print(test_data)
  for i in list(test_data):
    X_test.append(dictionary[i]) 
  pred_test=list(model_lstm.predict(np.array(X_test).reshape(1,99,1))[0])
  #print(len(pred_test))
  r=label_classes[pred_test.index(max(pred_test))]

  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  final_res=final_res+str(reverse_dictionary[round((r-min1)/(max1-min1),3)])
  test_data=test_data[-99:]+str(reverse_dictionary[round((r-min1)/(max1-min1),3)])


In [53]:
print("Generated characters from test data:")
print(final_res)

Generated characters from test data:
 wh say that the whrle of a teinn as a pensieceite shetgy of the poenent soeersannty oo she ooenenpes of the sorec oeraeption. and the poentsoes oe the poenestee anteerse aednnnticcted by the pajt the pajtc ane the same theng is an antertarid oatervel than the sare ald the soaneter tfe seeentes that the pajtc ase not aeloe and ane ane the semees of a teinn and the soaneter of the poenent soeersannty. bnt if the case of a sense ane anlears the sare oane is an antertariog of the poenent soeersannty of the poenent soeersannty of the poenent soeersannty of the pajtc as a poenent and the same theng is an antertarid oftsie aydee than the same theng is an antertarid oatervel thenry of the poenent soeersannty oo she ooenenpes of the sorec oeraeption. and the poentsoes oe the poenestee anteerse aednnnticcted by the pajt the pajtc ane the same theng is an antertarid oatervel than the sare ald the soaneter tfe seeentes that the pajtc ase not aeloe and ane ane 